<a href="https://colab.research.google.com/github/kwakseoyeon/test/blob/master/Schema_Mapping_Using_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports

In [ ]:
! pip install -qq langchain openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.3 MB/s eta 0:00:00


In [ ]:
import json
import os
from argparse import ArgumentParser

import pandas as pd
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks import wandb_tracing_enabled
from langchain.chat_models import ChatOpenAI

### Create dummy data

In [ ]:
source1 = """PatientID,FirstName,LastName,Gender,Age,DateOfBirth,Phone,Email
101,John,Smith,Male,35,15-07-1987,(555) 123-4567,john.smith@email.com
102,Mary,Johnson,Female,28,22-03-1995,(555) 987-6543,mary.j@email.com
103,David,Williams,Male,45,10-12-1978,(555) 555-5555,david.w@email.com
104,Sarah,Brown,Female,52,05-09-1971,(555) 111-2222,sarah.b@email.com
105,Michael,Davis,Male,30,20-11-1992,(555) 333-4444,michael.d@email.com
"""

source2="""PatientID,Name,Sex,Age,DOB,Telephone,Email
101,John Smith,M,35,15/07/1987,(555) 123-4567,john.smith@email.com
102,Mary Johnson,F,28,22/03/1995,(555) 987-6543,mary.j@email.com
103,David Williams,M,45,10/12/1978,(555) 555-5555,david.w@email.com
104,Sarah Brown,F,52,05/09/1971,(555) 111-2222,sarah.b@email.com
105,Michael Davis,M,30,20/11/1992,(555) 333-4444,michael.d@email.com
"""

sample="""ID,Full Name,Gender,Age,DOB,Mobile,Email
104,Sarah Brown,Female,52,1971-09-05,555-111-2222,sarah.b@email.com
105,Michael Davis,Male,30,1992-11-20,555-333-4444,michael.d@email.com
"""


In [ ]:
with open("source1.csv", "w") as fp:
  fp.write(source1)

with open("source2.csv", "w") as fp:
  fp.write(source2)

with open("sample.csv", "w") as fp:
  fp.write(sample)

### Load tables

In [ ]:
source1_df = pd.read_csv("source1.csv")
source1_df

,PatientID,FirstName,LastName,Gender,Age,DateOfBirth,Phone,Email
0,101,John,Smith,Male,35,15-07-1987,(555) 123-4567,john.smith@email.com
1,102,Mary,Johnson,Female,28,22-03-1995,(555) 987-6543,mary.j@email.com
2,103,David,Williams,Male,45,10-12-1978,(555) 555-5555,david.w@email.com
3,104,Sarah,Brown,Female,52,05-09-1971,(555) 111-2222,sarah.b@email.com
4,105,Michael,Davis,Male,30,20-11-1992,(555) 333-4444,michael.d@email.com


In [ ]:
source2_df = pd.read_csv("source2.csv")
source2_df

,PatientID,Name,Sex,Age,DOB,Telephone,Email
0,101,John Smith,M,35,15/07/1987,(555) 123-4567,john.smith@email.com
1,102,Mary Johnson,F,28,22/03/1995,(555) 987-6543,mary.j@email.com
2,103,David Williams,M,45,10/12/1978,(555) 555-5555,david.w@email.com
3,104,Sarah Brown,F,52,05/09/1971,(555) 111-2222,sarah.b@email.com
4,105,Michael Davis,M,30,20/11/1992,(555) 333-4444,michael.d@email.com


In [ ]:
sample_df = pd.read_csv("sample.csv")
sample_df

,ID,Full Name,Gender,Age,DOB,Mobile,Email
0,104,Sarah Brown,Female,52,1971-09-05,555-111-2222,sarah.b@email.com
1,105,Michael Davis,Male,30,1992-11-20,555-333-4444,michael.d@email.com


### Create a prompt

In [ ]:
template = """
Transform the data in Source 1 and Source 2 to match the format of the Sample row. Here are the contents of Source 1 and Source 2:

Source 1 - {source1_row}
Source 2 - {source2_row}

Please create a new row JSON object in the format of the Sample row:

Sample - {sample_row}

Pick the value from Source 1 or Source 2. Apply transformation so that the value changes to Sample format.
    JSON object:"""


### Query using Openai

In [ ]:
# SET YOUR OPENAI KEY HERE
OpenAI_API_KEY = ""

In [ ]:
prompt = PromptTemplate(
    template=template, input_variables=["source1_row", "source2_row", "sample_row"]
)

llm = ChatOpenAI(openai_api_key=OpenAI_API_KEY, model="gpt-3.5-turbo")
llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)

In [ ]:
openai_result_list = []
for i in range(len(source1_df)):
    source1_row = source1_df.iloc[i].to_json()
    source2_row = source2_df.iloc[i].to_json()
    sample_row = sample_df.iloc[-1].to_json()
    response = llm_chain.run(
        {"source1_row": source1_row, "source2_row": source2_row, "sample_row": sample_row, "sample_columns": sample_df.columns}
    )
    target_row_dict = json.loads(response)
    openai_result_list.append(target_row_dict)



> Entering new LLMChain chain...
Prompt after formatting:
 
Transform the data in Source 1 and Source 2 to match the format of the Sample row. Here are the contents of Source 1 and Source 2:

Source 1 - {"PatientID":101,"FirstName":"John","LastName":"Smith","Gender":"Male","Age":35,"DateOfBirth":"15-07-1987","Phone":"(555) 123-4567","Email":"john.smith@email.com"}
Source 2 - {"PatientID":101,"Name":"John Smith","Sex":"M","Age":35,"DOB":"15\/07\/1987","Telephone":"(555) 123-4567","Email":"john.smith@email.com"}

Please create a new row JSON object in the format of the Sample row:

Sample - {"ID":105,"Full Name":"Michael Davis","Gender":"Male","Age":30,"DOB":"1992-11-20","Mobile":"555-333-4444","Email":"michael.d@email.com"}

Pick the value from Source 1 or Source 2. Apply transformation so that the value changes to Sample format.
    JSON object:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
 
Transform the data in Source 1 and Source 2 to match the fo

In [ ]:
 target_df = pd.DataFrame(openai_result_list)
 target_df

,ID,Full Name,Gender,Age,DOB,Mobile,Email
0,101,John Smith,Male,35,1987-07-15,(555) 123-4567,john.smith@email.com
1,102,Mary Johnson,Female,28,1995-03-22,555-987-6543,mary.j@email.com
2,103,David Williams,Male,45,1978-10-12,555-555-5555,david.w@email.com
3,104,Sarah Brown,Female,52,1971-05-09,555-111-2222,sarah.b@email.com
4,105,Michael Davis,Male,30,1992-11-20,555-333-4444,michael.d@email.com


### Code Generation using GPT4

In [ ]:
template = """You are an assistant to generate code.

Lets think step by step

1. You are given three tables. Source1, Source2 and Sample.
2. Task is to generate a target table which has exactly the same number of columns as sample table and same number of rows as source1 table
3. For each column in the sample table, identify which column matches from source1 or source2 table and find the transformation needed from source to sample table
4. Use pandas in built functions or regex and transform the column into sample table format.
5. Apply mobile transformations(xxx-xxx-xxxx) simillar to the sample table format.
6. Always transform dates into yyyy-mm-dd format
7. Do not change the source1, source2 and sample table values. Instead, find the transformations and apply it on the target table.
8. Do not perform merge or concat, as the tables are huge.
9. The column names in the sample table might not match exactly in the source1 and source2 table. identify the columns based on the column values.
10. Generate python code to create target table by reading source1.csv, source2.csv, sample.csv.

Few rows of Source1, Source2 and Sample tables:

Source 1 - {source1_row}
Source 2 - {source2_row}
Sample - {sample_row}

Python Code:
"""

In [ ]:
prompt = PromptTemplate(
    template=template, input_variables=["source1_row", "source2_row", "sample_row"]
)

llm = ChatOpenAI(openai_api_key=OpenAI_API_KEY, model="gpt-4")
llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)

In [ ]:
source1_row = source1_df.iloc[:2].to_json()
source2_row = source2_df.iloc[:2].to_json()
sample_row = sample_df.iloc[:2].to_json()
response = llm_chain.run(
    {"source1_row": source1_row, "source2_row": source2_row, "sample_row": sample_row}
)
print(response)



> Entering new LLMChain chain...
Prompt after formatting:
You are an assistant to generate code. 

Lets think step by step

1. You are given three tables. Source1, Source2 and Sample.
2. Task is to generate a target table which has exactly the same number of columns as sample table and same number of rows as source1 table
3. For each column in the sample table, identify which column matches from source1 or source2 table and find the transformation needed from source to sample table
4. Use pandas in built functions or regex and transform the column into sample table format.
5. Apply mobile transformations(xxx-xxx-xxxx) simillar to the sample table format. 
6. Always transform dates into yyyy-mm-dd format
7. Do not change the source1, source2 and sample table values. Instead, find the transformations and apply it on the target table. 
8. Do not perform merge or concat, as the tables are huge.
9. The column names in the sample table might not match exactly in the source1 and source2 tab

In [ ]:
import pandas as pd
import re

# Load data
source1 = pd.read_csv('source1.csv')
source2 = pd.read_csv('source2.csv')
sample = pd.read_csv('sample.csv')

# Create a target DataFrame with the same number of rows as source1 and columns as sample
target = pd.DataFrame(index=range(len(source1)), columns=sample.columns)

# Map source column names to sample column names based on the content
column_map = {
    'PatientID': 'ID',
    'FirstName': 'Full Name',
    'LastName': 'Full Name',
    'Name': 'Full Name',
    'Gender': 'Gender',
    'Sex': 'Gender',
    'Age': 'Age',
    'DateOfBirth': 'DOB',
    'DOB': 'DOB',
    'Phone': 'Mobile',
    'Telephone': 'Mobile',
    'Email': 'Email'
}

# Apply transformations
for src_col, tgt_col in column_map.items():
    if src_col in source1.columns:
        if tgt_col == 'Full Name' and src_col in ['FirstName', 'LastName']:
            if tgt_col not in target.columns:
                target[tgt_col] = source1['FirstName'] + ' ' + source1['LastName']
            continue
        if tgt_col == 'Gender' and src_col == 'Sex':
            target[tgt_col] = source1[src_col].map({'M': 'Male', 'F': 'Female'})
            continue
        if tgt_col == 'DOB':
            target[tgt_col] = pd.to_datetime(source1[src_col], dayfirst=True).dt.strftime('%Y-%m-%d')
            continue
        if tgt_col == 'Mobile':
            target[tgt_col] = source1[src_col].apply(lambda x: re.sub(r'\D', '', x)).str.replace(r'(\d{3})(\d{3})(\d{4})', r'\1-\2-\3')
            continue
        target[tgt_col] = source1[src_col]
    elif src_col in source2.columns:
        if tgt_col == 'Full Name' and src_col == 'Name':
            target[tgt_col] = source2['Name']
            continue
        if tgt_col == 'Gender' and src_col == 'Sex':
            target[tgt_col] = source2[src_col].map({'M': 'Male', 'F': 'Female'})
            continue
        if tgt_col == 'DOB':
            target[tgt_col] = pd.to_datetime(source2[src_col], dayfirst=True).dt.strftime('%Y-%m-%d')
            continue
        if tgt_col == 'Mobile':
            target[tgt_col] = source2[src_col].apply(lambda x: re.sub(r'\D', '', x)).str.replace(r'(\d{3})(\d{3})(\d{4})', r'\1-\2-\3')
            continue
        target[tgt_col] = source2[src_col]

print(target)

    ID       Full Name  Gender  Age         DOB        Mobile  \
0  101      John Smith    Male   35  1987-07-15  555-123-4567   
1  102    Mary Johnson  Female   28  1995-03-22  555-987-6543   
2  103  David Williams    Male   45  1978-12-10  555-555-5555   
3  104     Sarah Brown  Female   52  1971-09-05  555-111-2222   
4  105   Michael Davis    Male   30  1992-11-20  555-333-4444   

                  Email  
0  john.smith@email.com  
1      mary.j@email.com  
2     david.w@email.com  
3     sarah.b@email.com  
4   michael.d@email.com  


<ipython-input-36-2b6cc9efe74c>:42: FutureWarning: The default value of regex will change from True to False in a future version.
  target[tgt_col] = source1[src_col].apply(lambda x: re.sub(r'\D', '', x)).str.replace(r'(\d{3})(\d{3})(\d{4})', r'\1-\2-\3')
<ipython-input-36-2b6cc9efe74c>:56: FutureWarning: The default value of regex will change from True to False in a future version.
  target[tgt_col] = source2[src_col].apply(lambda x: re.sub(r'\D', '', x)).str.replace(r'(\d{3})(\d{3})(\d{4})', r'\1-\2-\3')
